In [ ]:
# ======================
# IMPORTS
# ======================
import pandas as pd
import numpy as np
import torch
import json
import time
from itertools import combinations
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch_geometric.data import HeteroData
from tqdm.auto import tqdm
from scipy.spatial.distance import cdist
from joblib import Parallel, delayed
import re


# ----------------------------
# 1. Load the Excel Dataset
# ----------------------------
file_path = 'Incidents.xlsx'  # update with your file path
df = pd.read_excel(file_path)


In [ ]:


# Track missing values BEFORE imputation
missing_before = {
    "Job City (Before)": df['Job City'].isnull().sum(),
    "Device Address (Before)": df['Device Address'].isnull().sum()
}

# ------------------------------
# 2. CRITICAL FILTERS
# ------------------------------

# Only drop rows missing 'Job Substation'
df = df.dropna(subset=['Job Substation'])

# Convert date columns (errors become NaT)
df['Job OFF Time'] = pd.to_datetime(df['Job OFF Time'], errors='coerce')
df['Job ON Time']  = pd.to_datetime(df['Job ON Time'], errors='coerce')

# Keep rows where both times exist and OFF <= ON
time_mask = (
    df['Job OFF Time'].notna() &
    df['Job ON Time'].notna() &
    (df['Job OFF Time'] <= df['Job ON Time'])
)
df = df[time_mask]

# ----------------------------
# 3. Data Cleaning: Standardize Text Columns
# ----------------------------
# Standardize Job City and location fields (remove extra spaces, uppercase, etc.)
for col in ['Job City', 'Job Area (DISTRICT)', 'Job Substation', 'Job Feeder']:
    df[col] = df[col].astype(str).str.upper().str.strip()

# IMPORTANT: Replace string "NAN" with actual np.nan for Job City
#df['Job City'] = df['Job City'].replace('NAN', np.nan)
df['Job City'] = df['Job City'].replace(['NAN', '0'], np.nan)
# ----------------------------
# 4. Create a Composite Key
# ----------------------------
# We combine Job Area (DISTRICT), Job Substation, and Feeder ID (as a string) for maximum granularity.
#df['Feeder_ID_str'] = df['Feeder ID'].astype(str)  # convert numeric Feeder ID to string if needed
df['Feeder_ID_str'] = df['Feeder ID'].astype('Int64').astype(str).replace('<NA>', 'UNKNOWN')
df['CompositeKey'] = df['Job Area (DISTRICT)'] + "_" + df['Job Substation'] + "_" + df['Feeder_ID_str']

# Optionally, if you believe Job Feeder is more descriptive than Feeder ID, you can use:
# df['CompositeKey'] = df['Job Area (DISTRICT)'] + "_" + df['Job Substation'] + "_" + df['Job Feeder']

# ----------------------------
# 5. Impute Missing Job City Using the Composite Key
# ----------------------------
def get_mode(series):
    mode_series = series.mode()
    return mode_series.iloc[0] if not mode_series.empty else np.nan

# Build the mapping: for each CompositeKey, get the most common (mode) Job City among rows where it's not missing.
composite_city_map = (df[df['Job City'].notnull()]
                      .groupby('CompositeKey')['Job City']
                      .agg(get_mode)
                      .to_dict())

# Impute missing Job City using the composite key mapping
def impute_composite_job_city(row):
    if pd.isnull(row['Job City']) or row['Job City'] in ['NAN', '0']:  # Treat "0" as missing
        return composite_city_map.get(row['CompositeKey'], np.nan)
    return row['Job City']


df['Job City_imputed'] = df.apply(impute_composite_job_city, axis=1)

# ----------------------------
# 6. (Optional) Fallback for Job City Using Job Region
# ----------------------------
remaining_missing = df['Job City_imputed'].isnull().sum()
if remaining_missing > 0:
    region_to_city = (df[df['Job City_imputed'].notnull()]
                      .groupby('Job Region')['Job City_imputed']
                      .agg(get_mode)
                      .to_dict())
    def fallback_impute_job_city(row):
        if pd.isnull(row['Job City_imputed']):
            return region_to_city.get(row['Job Region'], np.nan)
        return row['Job City_imputed']
    df['Job City_imputed'] = df.apply(fallback_impute_job_city, axis=1)

# ----------------------------
# 7. Impute Missing Device Address Using a Similar Composite Approach
# ----------------------------
# Standardize Device Address
df['Device Address'] = df['Device Address'].astype(str).str.upper().str.strip()

# Replace the string "NAN" with actual np.nan
#df['Device Address'] = df['Device Address'].replace('NAN', np.nan)
df['Device Address'] = df['Device Address'].replace(['NAN', '0'], np.nan)

# Build the mapping: For each CompositeKey, get the most common Device Address (ignoring missing values)
composite_address_map = (df[df['Device Address'].notnull()]
                          .groupby('CompositeKey')['Device Address']
                          .agg(get_mode)
                          .to_dict())

def impute_composite_device_address(row):
    if pd.isnull(row['Device Address']) or row['Device Address'] in ['NAN', '0']:  # Treat "0" as missing
        return composite_address_map.get(row['CompositeKey'], np.nan)
    return row['Device Address']
    
df['Device Address_imputed'] = df.apply(impute_composite_device_address, axis=1)

# ----------------------------
# 8. (Optional) Drop Temporary Columns 
# ----------------------------
df = df.drop(columns=['Feeder_ID_str', 'CompositeKey'])

# ------------------------------
# 9. DROP NON-CRITICAL COLUMNS and Save the Result
# ------------------------------
cols_to_drop = [
    'Lead Crew Phone', 'AM Notes', 
    'Equipment Desc that should be excluded from reported indices',
    'Ark Grid Mod or OK Grid Enhancement Circuits'
]
#df = df.drop(columns=cols_to_drop, errors='ignore')

df.to_excel("Incidents_imputed.xlsx", index=False)

# ------------------------------
# 10. FINAL CHECK
# ------------------------------

# Track missing values AFTER imputation
missing_after = {
    "Job City (After)": df['Job City_imputed'].isnull().sum(),
    "Device Address (After)": df['Device Address_imputed'].isnull().sum()
}

# Display the difference (without displaying the entire dataset)
print("\n=== Missing Values Before & After Imputation ===")
for key in missing_before:
    before = missing_before[key]
    after = missing_after[key.replace("(Before)", "(After)")]
    reduction = before - after
    percent_reduction = (reduction / before * 100) if before > 0 else 0
    print(f"{key}: {before} → {after} (Reduced by {reduction} | {percent_reduction:.2f}%)")

# Final check for remaining missing values
print("\nFinal dataset shape:", df.shape)

# Display sample data (only the relevant columns)
print("\nSample of Job City and Device Address before & after imputation:")
print(df[['Job City', 'Job City_imputed', 'Device Address', 'Device Address_imputed']].head(10))


In [ ]:

import pandas as pd
import re

# Read the CSV files
substation_df = pd.read_csv("Substation.csv")
lines_df = pd.read_csv("Lines.csv")

# Standardize text columns (remove extra spaces, uppercase)
text_cols = ['Name', 'ADDRESSLIN', 'TOWN']
for col in text_cols:
    if col in substation_df.columns:
        substation_df[col] = substation_df[col].astype(str).str.upper().str.strip()

# Standardize text fields in Lines.csv (only LINENAME needed)
if 'LINENAME' in lines_df.columns:
    lines_df['LINENAME'] = lines_df['LINENAME'].astype(str).str.upper().str.strip()

print("✅ Step 1: Data read and standardized.")


In [ ]:
def extract_voltage(line_name):
    """Extract voltage from line name (e.g., '345kV Ranch Road')."""
    match = re.search(r'(\d+)kV', line_name)
    return int(match.group(1)) if match else None

# Extract voltages from Lines.csv
line_voltage_map = {}
for _, row in lines_df.iterrows():
    line_voltage = extract_voltage(row['LINENAME'])
    if not line_voltage:
        continue  # Skip lines where voltage couldn't be determined

    # Identify substations in the line name
    for substation in substation_df['Name']:
        if substation in row['LINENAME']:
            line_voltage_map[substation] = line_voltage  # Assign voltage

# Apply inferred voltages
substation_df['Voltage'] = substation_df.apply(
    lambda row: line_voltage_map.get(row['Name'], row['Voltage']), axis=1
)

print("✅ Step 2: Missing voltages inferred using transmission line data.")


In [ ]:
# Load incident data to extract feeder-based voltages
incident_df = pd.read_excel("Incidents_imputed.xlsx", engine='openpyxl')

def infer_voltage_from_feeders(substation_df, incident_df):
    """Infer missing substation voltages using feeder-based data from incidents."""
    
    # Build Feeder ID to Substation mapping
    feeder_map = incident_df.groupby('Feeder ID')['Job Substation'].apply(set).to_dict()
    
    # Dictionary to store inferred voltages
    feeder_voltages = {}

    for feeder, substations in feeder_map.items():
        # Get known voltages for substations in this feeder
        known_voltages = substation_df[substation_df['Name'].isin(substations)]['Voltage']
        valid_voltages = known_voltages[known_voltages > 0]  # Ignore zero voltages
        
        if not valid_voltages.empty:
            # Assign most common voltage for the feeder
            most_common_voltage = valid_voltages.mode()[0]  # Mode is the most frequent value
            for sub in substations:
                feeder_voltages[sub] = most_common_voltage

    # Apply inferred voltages
    substation_df['Voltage'] = substation_df.apply(
        lambda row: feeder_voltages.get(row['Name'], row['Voltage']), axis=1
    )

    return substation_df

# Apply voltage inference
substation_df = infer_voltage_from_feeders(substation_df, incident_df)
print("✅ Step 3: Remaining missing voltages inferred using feeder data.")


In [ ]:
# Check remaining missing or zero voltages
remaining_missing = substation_df[substation_df['Voltage'].isna() | (substation_df['Voltage'] == 0)].shape[0]

if remaining_missing > 0:
    print(f"⚠️ {remaining_missing} substations still have missing voltages after inference.")
else:
    print("✅ Step 4: All substations now have inferred voltages.")

# Save cleaned Substation data
substation_df.to_csv("Substation_Cleaned.csv", index=False)
print("✅ Cleaned Substation data saved as 'Substation_Cleaned.csv'.")

# Save standardized Lines data (No changes except name formatting)
lines_df.to_csv("Lines_Cleaned.csv", index=False)
print("✅ Standardized Lines data saved as 'Lines_Cleaned.csv'.")


In [ ]:
def infer_voltage_from_region(substation_df):
    """Infer missing voltage using the most common voltage in the same region."""
    
    # Get most common voltage per region
    region_voltage_map = (substation_df[substation_df['Voltage'] > 0]
                          .groupby('TOWN')['Voltage']
                          .agg(lambda x: x.mode()[0] if not x.mode().empty else None)
                          .to_dict())
    
    # Get most common voltage overall (fallback)
    most_common_voltage = substation_df[substation_df['Voltage'] > 0]['Voltage'].mode()[0]
    
    # Apply region-based inference
    substation_df['Voltage'] = substation_df.apply(
        lambda row: region_voltage_map.get(row['TOWN'], most_common_voltage) 
        if pd.isnull(row['Voltage']) or row['Voltage'] == 0 else row['Voltage'], axis=1
    )

    return substation_df

# Apply the function
substation_df = infer_voltage_from_region(substation_df)

# Verify again
remaining_missing = substation_df[substation_df['Voltage'].isna() | (substation_df['Voltage'] == 0)].shape[0]
if remaining_missing > 0:
    print(f"⚠️ {remaining_missing} substations STILL have missing voltages after region-based inference.")
else:
    print("✅ All substations now have voltage values after final inference.")

# Save the FINAL cleaned file
substation_df.to_csv("Substation_Cleaned_Final.csv", index=False)
print("✅ Final cleaned Substation data saved as 'Substation_Cleaned_Final.csv'.")



# # Preprocessing for Device Type Target Variable
# 
# This notebook performs the following steps:
# 
# 1. Loads the incident dataset.
# 2. Identifies and imputes missing values in the "Device Type" column using related columns ("Equip Desc" and "Dev Subtype").
# 3. Remaps mis-coded "0" values based on heuristic rules.
# 4. Defines a final mapping that preserves meaningful categories (Load, Fuse, Recloser, Switch, Breaker) and then merges ambiguous ones (Unknown, Source, Other) into Fuse (Option 2).
# 5. Maps the final string labels to integer labels.
# 6. Saves the cleaned DataFrame to an Excel file for use in your main pipeline.


In [ ]:
import pandas as pd
import numpy as np
import re
from IPython.display import display

# ======================
# 1. Data Loading
# ======================
incident_df = pd.read_excel('Incidents_imputed.xlsx', engine='openpyxl')
print("Initial data shape:", incident_df.shape)

In [ ]:
# ======================
# 2. Device Type Imputation
# ======================
# Create mappings from existing data
equip_mapping = (
    incident_df.dropna(subset=['Device Type'])
    .groupby('Equip Desc')['Device Type']
    .agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
)

subtype_mapping = (
    incident_df.dropna(subset=['Device Type'])
    .groupby('Dev Subtype')['Device Type']
    .agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
)

# Imputation function with priority logic
def impute_device_type(row):
    if pd.notna(row['Device Type']):
        return row['Device Type']
    if row['Equip Desc'] in equip_mapping:
        return equip_mapping[row['Equip Desc']]
    if row['Dev Subtype'] in subtype_mapping:
        return subtype_mapping[row['Dev Subtype']]
    return np.nan  # Temporary placeholder

incident_df['Device Type_imputed'] = incident_df.apply(impute_device_type, axis=1)

In [ ]:
# ======================
# 3. Handle Special Cases ("0" values)
# ======================
def map_zero_device_type(equip_desc):
    """Improved mapping with regex patterns"""
    desc = str(equip_desc).lower()
    
    if re.search(r'\bfuse\b', desc):
        return 'Fuse'
    elif re.search(r'\brecloser\b', desc):
        return 'Recloser' 
    elif re.search(r'\bswitch\b', desc):
        return 'Switch'
    elif re.search(r'\bload\b', desc):
        return 'Load'
    elif re.search(r'\bbreaker\b', desc):
        return 'Breaker'
    else:
        return 'Ambiguous'

# Apply to rows with "0" device type
zero_mask = incident_df['Device Type_imputed'].astype(str).str.strip() == "0"
incident_df.loc[zero_mask, 'Device Type_imputed'] = incident_df.loc[zero_mask, 'Equip Desc'].apply(map_zero_device_type)

In [ ]:
# ======================
# 4. Final Class Mapping
# ======================
# Define authoritative categories
MAIN_CATEGORIES = ['Load', 'Fuse', 'Recloser', 'Switch', 'Breaker']

def categorize_device(x):
    """Smart categorization with validation"""
    if x in MAIN_CATEGORIES:
        return x
    elif x == 'Ambiguous':
        # Merge ambiguous cases with smallest existing category
        class_counts = incident_df['Device Type_imputed'].value_counts()
        smallest_class = class_counts[class_counts.index.isin(MAIN_CATEGORIES)].idxmin()
        return smallest_class
    else:
        return 'Other'

incident_df['Device_Final'] = incident_df['Device Type_imputed'].apply(categorize_device)

In [ ]:
# ======================
# 5. Validation & Quality Checks
# ======================
print("\n=== Class Distribution ===")
dist = incident_df['Device_Final'].value_counts(normalize=True)
display(dist)

print("\n=== Imputation Accuracy Check ===")
# Check original vs imputed values where both exist
valid_mask = incident_df['Device Type'].notna() & incident_df['Device_Final'].notna()
accuracy = (incident_df.loc[valid_mask, 'Device Type'] == incident_df.loc[valid_mask, 'Device_Final']).mean()
print(f"Consistency with original data: {accuracy:.1%}")

In [ ]:
# ======================
# 6. Final Output
# ======================
label_map = {cat: idx for idx, cat in enumerate(MAIN_CATEGORIES)}
incident_df['Device_Label'] = incident_df['Device_Final'].map(label_map)

# Save processed data
incident_df.to_excel('Incident_multi_class_processed.xlsx', index=False)
print("\nProcessing complete. Final labels:")
print(incident_df['Device_Label'].value_counts())

In [ ]:
###
import pandas as pd
import numpy as np
import re
from IPython.display import display

# ======================
# 1. Data Loading
# ======================
incident_df = pd.read_excel('Incidents_imputed.xlsx', engine='openpyxl')
print("Initial data shape:", incident_df.shape)

# ======================
# 2. Unique Values & Distribution for Equip Desc
# ======================
# Get frequency distribution for Equip Desc
equip_desc_counts = incident_df['Equip Desc'].value_counts()
print("Number of unique Equip Desc values:", equip_desc_counts.shape[0])
print("\nEquip Desc Distribution:")
display(equip_desc_counts)

# Optionally, display the sorted list of unique Equip Desc values for inspection:
unique_equip_desc = sorted(incident_df['Equip Desc'].dropna().unique())
print("\nSorted list of unique Equip Desc values:")
display(unique_equip_desc)


In [2]:
misc_df = incident_df[incident_df['Equipment_Group'] == 'Miscellaneous']  
print("Top Terms in 'Miscellaneous':")  
print(misc_df['Equip Desc'].value_counts().head(20))  

Top Terms in 'Miscellaneous':
Equip Desc
CUSTOMER EQUIPMENT       22818
SUBSTATION CIRCUIT       11790
OTHER                    11364
POLE                     11071
CONNECTOR                 6482
JUMPER                    5907
METER - SECONDARY         1935
CROSSARM                  1782
LINE RECLOSER             1408
ARRESTOR OH               1392
INSULATOR                 1282
SCADA - CIRCUIT TRIP      1188
SCADA - LOCKOUT           1111
PEDESTAL/J-BOX             910
PIN                        525
BUSHING                    294
CUSTOMER CONNECTIVITY      180
ELBOW                      169
METER - PRIMARY            160
SPLICE - OH                139
Name: count, dtype: int64


In [3]:
import pandas as pd
import re

# Load data
incident_df = pd.read_excel('Incidents_imputed.xlsx', engine='openpyxl')

# Step 1: Remove noise
noise_terms = ['CANCELLED', 'ON UPON ARRIVAL', 'OTHER']
incident_df = incident_df[~incident_df['Equip Desc'].isin(noise_terms)]

# Step 2: Enhanced functional grouping
EQUIP_GROUP_MAPPING = {
    # Core Electrical Components
    r'\bFUSE\b': 'Fuse',
    r'\bCUTOUT\b': 'Fuse',
    r'\b(RECLOSER|ARRESTER|ARRESTOR)\b': 'Protection_Device',
    r'\bTRANSFORMER\b': 'Transformer',
    
    # Conductors & Connections
    r'\bCONDUCTOR\b': 'Conductor',
    r'\b(SPLICE|CONNECTOR|JUMPER)\b': 'Connector',
    
    # Structural Components
    r'\b(POLE|CROSSARM|PIN|BUSHING)\b': 'Structural',
    
    # Customer-Related
    r'\bCUSTOMER\b': 'Customer_Equipment',
    
    # Monitoring & Control
    r'\bSCADA\b': 'Monitoring_System',
    r'\bMETER\b': 'Meter',
    
    # Power Regulation
    r'\bREGULATOR\b': 'Voltage_Regulator',
    r'\bCAPACITOR\b': 'Power_Factor_Correction'
}

def categorize_equip(desc):
    desc = str(desc).upper()
    for pattern, group in EQUIP_GROUP_MAPPING.items():
        if re.search(pattern, desc, flags=re.IGNORECASE):
            return group
    return 'Other'

incident_df['Equipment_Group'] = incident_df['Equip Desc'].apply(categorize_equip)

# Step 3: Strategic merging of small groups
incident_df['Equipment_Group'] = incident_df['Equipment_Group'].replace({
    'Voltage_Regulator': 'Power_Management',
    'Power_Factor_Correction': 'Power_Management',
    'Meter': 'Monitoring_System',
    'Structural': 'Infrastructure'
})

# Final class consolidation
incident_df['Equipment_Group'] = incident_df['Equipment_Group'].replace({
    'Power_Management': 'Other',          # 2.8% → Other
    'Monitoring_System': 'Other'          # 1.9% → Other
})

# Verify distribution
print("\nFinal Class Distribution:")
dist = incident_df['Equipment_Group'].value_counts(normalize=True)
print(dist)


Final Class Distribution:
Equipment_Group
Fuse                  0.425505
Conductor             0.207312
Customer_Equipment    0.099457
Other                 0.090955
Infrastructure        0.059268
Connector             0.054395
Transformer           0.050524
Protection_Device     0.012585
Name: proportion, dtype: float64


In [6]:
import pandas as pd  
import re  

# Load data  
incident_df = pd.read_excel('Incidents_imputed.xlsx', engine='openpyxl')  

# Step 1: Remove noise  
noise_terms = ['CANCELLED', 'ON UPON ARRIVAL', 'OTHER']  
incident_df = incident_df[~incident_df['Equip Desc'].isin(noise_terms)]  

# Step 2: Enhanced regex patterns  
EQUIP_GROUP_MAPPING = {  
    r'\bFUSE\b': 'Fuse',  
    r'\bCUTOUT\b': 'Fuse',  
    r'\bCONDUCTOR\b': 'Conductor',  
    r'\b(RECLOSER|ARRESTOR|SUBSTATION CIRCUIT|RELAY|CIRCUIT BREAKER|GROUNDING)\b': 'Protection_Device',  
    r'\b(TRANSFORMER|XFMR)\b': 'Transformer',  
    r'\b(CONNECTOR|JUMPER|SPLICE)\b': 'Infrastructure',  
    r'\bCUSTOMER\b': 'Customer_Equipment',  
    r'\b(POLE|CROSSARM|PIN|TOWER|ANCHOR)\b': 'Infrastructure',  
    r'\b(REGULATOR|CAPACITOR)\b': 'Power_Management'  
}  

def categorize_equip(desc):  
    desc = str(desc).upper()  
    for pattern, group in EQUIP_GROUP_MAPPING.items():  
        if re.search(pattern, desc, flags=re.IGNORECASE):  
            return group  
    return 'Customer_Equipment'  # Default to customer-facing issues  

incident_df['Equipment_Group'] = incident_df['Equip Desc'].apply(categorize_equip)  

# Step 3: Final merging  
incident_df['Equipment_Group'] = incident_df['Equipment_Group'].replace({  
    'Power_Management': 'Infrastructure',  
    'Transformer': 'Infrastructure'  
})  

# Verify  
print("\nFinal Distribution:")  
print(incident_df['Equipment_Group'].value_counts(normalize=True))  


Final Distribution:
Equipment_Group
Fuse                  0.425505
Conductor             0.207312
Infrastructure        0.164330
Customer_Equipment    0.139265
Protection_Device     0.063589
Name: proportion, dtype: float64
